## Cross-Platform verification

In this tutorial, we will show to measure the fidelity $\mathcal{F}=\mathrm{Tr}(\rho_1\rho_2)/\sqrt{\mathrm{Tr}(\rho_1^2)\mathrm{Tr}(\rho_2^2)}$ between two states $\rho_1$, $\rho_2$ prepared on two quantum devices. 

Related papers [Elben et al, PRL 2019](https://doi.org/10.1103/PhysRevLett.124.010504), [Zhu et al, Nat. Comm 2022](https://www.nature.com/articles/s41467-022-34279-5)

In [1]:
using RandomMeas

We first consider a state $\rho_1$ prepared by a random quantum circuit

In [2]:
N = 6
ξ = siteinds("Qubit", N)
depth = 2
circuit = random_circuit(ξ, depth)
states = ["Dn" for n in 1:N]
ψ0 = MPS(ξ,states);
ρ0 = outer(ψ0',ψ0);

# Prepare state 1
ρ1 = apply(circuit,ρ0,apply_dag=true);
println("State 1 prepared")

# Prepare state 2
p = 0.1*rand(N)
ρ2 = apply_depo_channel(ρ1,p)
println("State 2 prepared")


State 1 prepared
State 2 prepared


In [3]:
overlap = real(inner(ρ1,ρ2))
purity1 = get_purity(ρ1)
purity2 = get_purity(ρ2)
F = overlap/max(purity1,purity2)
println("Overlap ",overlap)
println("Purity1 ",purity1)
println("Purity2 ",purity2)
println("Fidelity ",F)

Overlap 0.8216351964933482
Purity1 0.9999999999999984
Purity2 0.6801920800798407
Fidelity 0.8216351964933495


We then sample randomized measurement settings (local random unitaries)

In [4]:
measurement_settings = LocalUnitaryMeasurementSettings(N,500,site_indices=ξ);

We simulate the experiment by sampling randomized measurement on state ρ1.

In [5]:
measurement_data_1 = simulate_local_measurements(ρ1,200,mode="dense",measurement_settings=measurement_settings);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:04


In our simulated experiment, the second state $\rho_2$ differs from $\rho_1$ due to local depolarization. We sample randomized measurements according to the same random unitaries $u$. For some reason, experiment 2 is a bit faster than experiment 1. Thus, we choose a higher number of measurements per unitary, $N_M = 500$.

In [6]:
measurement_data_2 = simulate_local_measurements(ρ2,500,mode="dense",measurement_settings=measurement_settings);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


We then use the formula $\mathrm{tr}(\rho_1\rho_2)=(-2)^{-D[s,s']}\sum_{s,s'}P_u^{(1)}(s)P_u^{(2)}(s')$ to extract the overlap (and proceed similarly to access the purities $\mathrm{tr}(\rho_1^2)$, $\mathrm{tr}(\rho_2^2)$). In this process, the Born probabilities for each random unitary are computed. Thus, the process is not measurment efficient.

In [7]:
overlap = get_overlap_direct(measurement_data_1,measurement_data_2)
println("Estimated overlap = ",overlap)
purity_1 = get_purity_direct(measurement_data_1)
println("Estimated purity 1 = ",purity_1)
purity_2 = get_purity_direct(measurement_data_2)
println("Estimated purity 2 = ",purity_2)
Fs= overlap/max(purity_1,purity_2)
println("Estimated fidelity ",Fs)

Estimated overlap = 0.7809328000000002
Estimated purity 1 = 0.9682527638190956
Estimated purity 2 = 0.6424444408817631
Estimated fidelity 0.8065381573709677
